In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
from datetime import datetime
import time
import math
import sys
import pandas as pd

In [ ]:
critical_priority = [
"priority: blocker",
# "criticalpriority",
"priority-critical",
"critical priority",
"priority:critical",
"priority critical",
"priority: critical",
"priority - critical",
"critical-priority",
"priority/urgent",
"priority/blocker",
"priority/critical",
"priority/important",
"critical",
"important",
"urgent"
]

In [ ]:
high_priority = [
"priority: major",
"highpriority",
"priority-high",
"high priority",
"priority:high",
"priority high",
"priority: high",
"priority - high",
"high-priority",
"priority/high",
"is:priority"
]

In [ ]:
medium_priority = [
"priority:normal",
"mediumpriority",
"priority-medium",
"medium priority",
"priority:medium",
"priority medium",
"priority: medium",
"priority - medium",
"medium-priority",
"priority/medium",
"priority/normal",
"priority: middle"
]

In [ ]:
low_priority = [
"priority: minor",
"lowpriority",
"priority-low",
"low priority",
"priority:low",
"priority low",
"priority: low",
"priority - low",
"low-priority",
"priority/low",
"is:no-priority"
]

In [ ]:
wontfix = [
"priority: trivial",
"won't fix",
"wont fix",
"wontfix",
"wont-fix",
"status: won't fix",
"will not fix",
"resolution:won't fix",
"status=will-not-fix",
"closed: won't fix",
"state:wont-fix",
"status: will not fix",
"won't-fix",
"will-not-fix",
"cant-fix",
"cantfix",
"can't fix",
"rejected"
]

In [ ]:
all_labels = {"critical_priority":critical_priority, "high_priority":high_priority, "medium_priority":medium_priority, "low_priority":low_priority, "wontfix":wontfix}

In [ ]:
delay_conn = 60
clients = [('your_clinet_id_1', 'your_clinet_secret_1'),
           ('your_clinet_id_2', 'your_clinet_secret_2'),
           ('...', '...')]

clients_number = len(clients)          
client_index = 0
headers = {}
headers['User-Agent'] = "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17"
headers['Accept'] = 'Accept: application/vnd.github.squirrel-girl-preview'

def get_data_pages(req_url, upper_bound=math.inf, key=-1):    
    page_number = 1
    resp_list = [] 
    
    while(page_number<=upper_bound):         
        try:   
            r = requests.get(req_url + "&page=" + str(page_number), headers=headers) 
            if(r.ok):                            
                result = json.loads(r.text or r.content)
                
                if key == -1: 
                    data = result                        
                else:
                    if not key in result:
                        print("key not exists")    
                        break                    
                    data = result[key]

                resp_list += data          
                if not data:
                    break
                page_number += 1
                    
                try:
                    if int(r.headers["X-RateLimit-Remaining"]) < 2:
                        print("limit exceeded!!!!!!!!!!!!")
                        delay = 60
                        print('sleeping for '+str(delay)+' seconds...')
                        print("current time:" + str(datetime.now()))
                        time.sleep(int(delay))
                except (KeyError):
                    pass            
            else:
                resp = json.loads(r.text or r.content)
                print('\n---'+str(r))
                print('\n---'+str(resp['message']))
                return False
        except requests.exceptions.Timeout as e:
            print("-------timeout-------")
            print(e)
            time.sleep(delay_conn)
            return get_data_pages(req_url, upper_bound, key)
        except requests.ConnectionError as e:
            print("-------connection error-------")
            print(e)
            time.sleep(delay_conn)
            return get_data_pages(req_url, upper_bound, key)
    return resp_list

In [ ]:
counter = clients_number*3  #in each 1000 issue batch, we request 10 pages, so we can crawl 3 batch per minute with each client
start_time = datetime.now()

for label_cat in ["critical_priority", "wontfix"]:
    cat_issues = []
    for label in all_labels[label_cat]:
        start_time = datetime.now() 
        last_date = '2020-11-10T10:23:19Z'
        issues_list = []
        batch_number = 1
        while True:    
            client_id, client_secret = clients[client_index]
            issues_req = f'https://api.github.com/search/issues?q=label:"{label}"+state:closed+created:<{last_date}+language:java&sort=created&per_page=100&client_id={client_id}&client_secret={client_secret}'
            batch_issues_list = get_data_pages(issues_req, 10, "items")

            if not batch_issues_list:
                break

            client_index += 1
            if(client_index == clients_number):
                client_index = 0

            last_date = batch_issues_list[-1]["created_at"]
            issues_list += batch_issues_list
            batch_number += 1
            
            counter -= 1
            if counter == 0:
                end_time = datetime.now()
                duration = (end_time - start_time).seconds
                if(duration < 60):
                    time.sleep(60 - duration)
                    print(f"sleep for {60 - duration} seconds")
            counter = clients_number*3
            start_time = datetime.now()

        print(label, len(issues_list))
        if(issues_list == []):
            print(f"there is no issue for label {label}")
            
        cat_issues += issues_list

    with open(f"data/{label_cat}_issues.txt", "w") as f:
        f.write(json.dumps(cat_issues, indent=4))

print("\n*********finished*********")

In [ ]:
for label_cat in all_labels:
    with open(f"data/{label_cat}_issues.txt") as f:
        obj = json.loads(f.read())
        print(label_cat, len(obj))

In [ ]:
labelcat_repos = {}
for label_cat in all_labels:
    with open(f"data/{label_cat}_issues.txt") as f:
        obj = json.loads(f.read())
    labelcat_repos[label_cat] = [i["repository_url"] for i in obj]

In [ ]:
# p123 = list(set([repo for repo in label_repos["p1"] if repo in label_repos["p2"] and repo in label_repos["p3"]]))

all_repos = [] 
for label_cat in all_labels:
    all_repos += labelcat_repos[label_cat]
all_repos = list(set(all_repos))

print(len(all_repos))

In [ ]:
p_count = {label_cat:[] for label_cat in all_labels}

In [ ]:
for repo in all_repos:
    for label_cat in all_labels:
        p_count[label_cat].append(labelcat_repos[label_cat].count(repo))

In [ ]:
df = pd.DataFrame(columns = ["repo"] + [label_cat for label_cat in all_labels])

In [ ]:
df["repo"] = all_repos
for label_cat in all_labels:
    df[label_cat] = p_count[label_cat]

In [ ]:
df.to_csv("data/repos_priority_labels.csv", index=False)